In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import special
from sklearn import linear_model
from mpmath import mp
import parametric_lasso 
import util 
import crossvalidation_event 
import gen_data
import ci 
import warnings
from statsmodels.othermod.betareg import BetaModel

In [2]:
# Generate data
def gen_Xy(n,beta_vec,phi_true):
    p = len(beta_vec)-1
    X = np.column_stack((np.ones(n), np.random.normal(0,1,n*p).reshape(n,p)))
    eta_true = np.dot(X,beta_vec)
    mu_true = np.exp(eta_true)/(1 + np.exp(eta_true))
    y = np.random.beta(mu_true*phi_true,(1-mu_true)*phi_true,n)
    return X, y

In [3]:
# MLE
def betareg(X,y,tol=1e-6,kmax=20):

    # get dimension of X
    dm = X.shape
    n = dm[0]
    p = dm[1] 

    # bound y away from 0 and 1 by 1e-6 for the sake of numerical stability
    y = (y < 1e-6) * 1e-6 + (y > (1 - 1e-6)) * (1 - 1e-6) + y *( (y > 1e-6) & (y < (1 - 1e-6)))
    
    # compute transformed response values
    y_tilde = np.log(y/(1-y))
    
    # initialize b and phi
    b = np.zeros(p)
    phi = 1
    
    # update b and phi until convergence
    conv = 0
    k = 1
    while conv == 0 and k < kmax:
        
        b0 = b

        eta = np.dot(X,b)
        mu = np.exp(eta)/(1 + np.exp(eta))
        d = mu*(1-mu)
        w = d * np.sqrt(phi * (special.polygamma(1,mu*phi) + special.polygamma(1,(1-mu)*phi)))
        mu_tilde = special.polygamma(0,mu*phi) - special.polygamma(0,(1-mu)*phi)
        
        # update b while holding phi fixed
        U = np.multiply(w.reshape((n,1)),X)
        z = np.dot(U,b) + d / w * (y_tilde - mu_tilde)
        b = np.linalg.lstsq(U,z,rcond=None)[0]
    
        # update phi while holding b fixed
        dphi = n*special.polygamma(0,phi) + np.sum(mu*(y_tilde - mu_tilde) + np.log(1-y) - special.polygamma(0,(1-mu)*phi))
        d2phi = n*special.polygamma(1,phi) - np.sum(mu**2 * special.polygamma(1,mu*phi) + (1-mu)**2 * special.polygamma(1,(1-mu)*phi))
        phi = phi - dphi / d2phi

        # check convergence and increment k
        conv = max(abs(b-b0)) < tol
        k = k+1

    return b, phi, U, z

In [4]:
# Parametric programming CV for linear regression
def linreg_pp_cv(X,y,list_lam, train,cov_scale):
    
    threshold = 20
    cov = np.identity(n) * cov_scale # so for beta regression put in cov_scale = 1/phi
    
    cutoff = int(train * n )
    
    X_train = X[:cutoff, :]
    y_train = y[:cutoff]
    
    X_val = X[cutoff:n, :]
    y_val = y[cutoff:n]
    
    min_cv_error = np.Inf
    lam = None
    lam_idx = None
    
    for i in range(len(list_lam)):
        
        each_lam = list_lam[i]
        clf_lam = linear_model.Lasso(alpha=each_lam, fit_intercept=False)
        clf_lam.fit(X_train, y_train)
        bh_lam = clf_lam.coef_
        bh_lam = bh_lam.reshape((len(bh_lam), 1))
        temp_cv_error = 0.5*sum((y_val - (np.dot(X_val, bh_lam)).flatten())**2)
        
        if temp_cv_error < min_cv_error:
            min_cv_error = temp_cv_error
            lam = each_lam
            lam_idx = i
    
    best_lam = list_lam[lam_idx]
    clf = linear_model.Lasso(alpha=lam, fit_intercept=False)
    clf.fit(X, y)
    bh = clf.coef_
    
    y = y.reshape((n, 1))
    
    A, XA, Ac, XAc, bhA = util.construct_A_XA_Ac_XAc_bhA(X, bh, n, p)

    p_val = [None]*p
    CI_lo_005 = [None]*p
    CI_up_005 = [None]*p
    
    for j_selected in A:
    
        etaj, etajTy = util.construct_test_statistic(j_selected, XA, y, A)
    
        a, b = crossvalidation_event.compute_a_b(y, etaj, n)
        a_flatten = a.flatten()
        b_flatten = b.flatten()
        a_train = (a_flatten[:cutoff]).reshape((cutoff, 1))
        b_train = (b_flatten[:cutoff]).reshape((cutoff, 1))
    
        a_val = (a_flatten[cutoff:n]).reshape((n - cutoff, 1))
        b_val = (b_flatten[cutoff:n]).reshape((n - cutoff, 1))
    
        list_zk_min_lam, list_bhz_min_lam, list_active_set_min_lam, list_etaAkz_min_lam, list_bhAz_min_lam = \
            parametric_lasso.run_parametric_lasso_cv(X_train, list_lam[lam_idx], X_train.shape[0], p, threshold, a_train, b_train)
    
        piecewise_quadratic_min_lam = crossvalidation_event.construct_piecewise_quadratic(a_val, b_val, X_val, list_zk_min_lam,
                                                                      list_active_set_min_lam, list_etaAkz_min_lam,
                                                                      list_bhAz_min_lam)
    
        set_piecewise_funct = [piecewise_quadratic_min_lam]
        set_list_zk = [list_zk_min_lam]
    
        for i in range(len(list_lam)):
            if i == lam_idx:
                continue
    
            list_zk_i, list_bhz_i, list_active_set_i, list_etaAkz_i, list_bhAz_i = \
                parametric_lasso.run_parametric_lasso_cv(X_train, list_lam[i], X_train.shape[0], p, threshold, a_train, b_train)
    
            piecewise_quadratic_i = crossvalidation_event.construct_piecewise_quadratic(a_val, b_val, X_val, list_zk_i,
                                                                  list_active_set_i, list_etaAkz_i, list_bhAz_i)
    
            set_piecewise_funct.append(piecewise_quadratic_i)
            set_list_zk.append(list_zk_i)
    
        z_interval_cv = crossvalidation_event.construct_z_interval_cv(set_piecewise_funct, set_list_zk)
    
        list_zk, list_bhz, list_active_set = parametric_lasso.run_parametric_lasso(X, y, lam, etaj, n, p, threshold)
    
        z_interval_m = crossvalidation_event.construct_m_z_interval(A, list_active_set, list_zk)
    
        z_interval = crossvalidation_event.construct_z_interval(z_interval_m, z_interval_cv)
    
        pivot = util.pivot_with_specified_interval(z_interval, etaj, etajTy, cov, 0)

        if pivot is None:
            p_val[j_selected] = None
        else: 
            p_val[j_selected] = 2 * min(1 - pivot, pivot)

        confidence_interval_005 = ci.compute_ci_with_specified_interval(z_interval, etaj, etajTy, cov, bh[j_selected], 0.05)
        if confidence_interval_005 is None:
            CI_lo_005[j_selected] = None
            CI_up_005[j_selected] = None
        else:
            CI_lo_005[j_selected] = confidence_interval_005[0]
            CI_up_005[j_selected] = confidence_interval_005[1]
        
    return best_lam, A, bh, p_val, CI_lo_005, CI_up_005

In [5]:
# Parametric programming CV for beta regression
def betareg_pp_cv(X,y,list_lambda,train):
    
    # fit the beta regression
    bmle, phimle, Umle, zmle = betareg(X,y)
    
    # remove effect of the intercept column from both zmle and Umle
    u0 = Umle[:,0]
    z = zmle - u0*bmle[0]
    Pu0 = 1 / sum(u0**2) * np.outer(u0,u0.T)
    U = np.delete(Umle,0,1)
    U = U - np.matmul(Pu0,U)
    
    # feed U and z as well as 1/phi into the linreg_pp_cv function
    best_lam, A, bh, p_val, CI_lo_005, CI_up_005 = linreg_pp_cv(U,z,list_lambda,train,cov_scale = 1/phimle)

    return best_lam, A, bh, p_val, CI_lo_005, CI_up_005

### Simulation

In [6]:
n = 500
p = 20
beta_vec = [-2,1,-1/2,1/2] + [0]*17
phi_true = 10

lambdas_lo = 0.002
lambdas_hi = 0.010
n_lambdas = 20
lambdas = np.logspace(np.log10(lambdas_lo), np.log10(lambdas_hi), num = n_lambdas)
train = 0.7

X, y = gen_Xy(n=n, beta_vec=beta_vec, phi_true = phi_true)
best_lam, A, bh, p_val, CI_lo, CI_up = betareg_pp_cv(X=X, y=y, list_lambda=lambdas, train=train)

In [7]:
lambdas

array([0.002     , 0.0021768 , 0.00236922, 0.00257866, 0.00280661,
       0.00305471, 0.00332474, 0.00361864, 0.00393852, 0.00428668,
       0.00466562, 0.00507805, 0.00552694, 0.00601551, 0.00654727,
       0.00712604, 0.00775597, 0.00844159, 0.00918781, 0.01      ])

In [8]:
best_lam

0.002578657607116292

In [9]:
p_val

[0.0,
 0.0016262195515221991,
 0.0,
 0.9464515401472182,
 0.9751194794556318,
 0.49086846701149617,
 None,
 None,
 0.24855679241855125,
 None,
 None,
 0.6490709907117287,
 0.40833070308295305,
 None,
 0.2439821572555217,
 0.33002232107754803,
 0.8042033680986491,
 0.14288698742456368,
 0.8747192857440703,
 0.2491136188019473]